In [1]:
# this code generate synthetic dataset for machine learning
import sklearn.datasets
import numpy as np
import pandas as pd
import math
from random import randint
import scipy.stats as ss
import copy

# AutoSKLearn imports
import autosklearn.classification
from statistics import mean
from sklearn.svm import LinearSVC


/home/tliu/.conda/envs/shap/lib/python3.7/site-packages/pyparsing.py:2910: FutureWarning: Possible set intersection at position 3
  self.re = re.compile( self.reString )


In [2]:
# define parameters
n_feat = 3000
n_sample = 300
n_query = 20
n_substitute = 2
t_query = 2; #time budget given to each query (both informative-part-search and actual query)
fold = 3

threshold1 = 0.1 # For applying thresholdout to panel screening 
noise1 = 0.03 #scale of laplacian noise used for determining if a part is informative
threshold2 = 0.05; #Thresholdout only! used for determining if query result is acc_test or acc_train
noise2 = 0.01; #Thresholdout only! scale of laplacian noise used for determining if query result is acc_test or acc_train
noise3 = 0.01; #Thresholdout only! scale of laplacian noise applied in reported query result

In [3]:
# Make synthetic data 1: 10000fet~300samples (100train 100test and 100fresh test), 100 infromative features
# a = sklearn.datasets.make_classification(n_samples=n_sample, n_features=n_feat, n_informative=10, n_redundant=0, n_repeated=0, n_classes=2, n_clusters_per_class=1, weights=None, flip_y=0.25, class_sep=4.0, hypercube=True, shift=None, scale=None, shuffle=True, random_state=None)
# X = a[0] # data part of the dataset
# X_train =X[0:100] # training set contains 100 samples
# X_test =X[100:200] # testing set contains 100 samples
# X_fresh =X[200:300] # fresh test set contains 100 samples
# y = a[1] # label of all the samples
# y_train = np.asarray(y[0:100]) # training set contains 100 samples
# y_test = np.asarray(y[100:200]) # testing set contains 100 samples
# y_fresh = np.asarray(y[200:300]) # fresh test set contains 100 samples
# X_train = pd.DataFrame(X_train) # transfer to pandas dataframe
# X_test = pd.DataFrame(X_test)
# X_fresh = pd.DataFrame(X_fresh)

In [4]:
# Make synthetic data 2: 10000fet~300samples(100train 100test and 100fresh test), no infromative features
mu, sigma = 0, 1 # mean and standard deviation
X = []; # data portion of the synthetic dataset
for x in range(0, n_feat):
    s = np.random.normal(mu, sigma, n_sample)
    X.append(s)
X = pd.DataFrame(X)
X = X.T
X_train =X[0:int(n_sample/3)] # training set contains 1/3 samples
X_test =X[int(n_sample/3):int(2*n_sample/3)] # testing set contains 1/3 samples
X_fresh =X[int(2*n_sample/3):int(n_sample)] # fresh test set contains 1/3 samples
y = np.random.normal(mu, sigma, n_sample) # create a evenly balanced label (of 0,1) of length 300
# import matplotlib.pyplot as plt 
# count, bins, ignored = plt.hist(y, 30, normed=True)
# plt.plot(bins, 1/(sigma * np.sqrt(2 * np.pi)) *np.exp( - (bins - mu)**2 / (2 * sigma**2) ),linewidth=2, color='r')
# plt.show()
y[y>=0] = 1 #50% with label 1
y[y<0] = 0 #50% with label 0
y_new = [int(i) for i in y] # convert label to integer
y_train = np.asarray(y_new[0:int(n_sample/3)]) # training set contains 1/3 samples
y_test = np.asarray(y_new[int(n_sample/3):int(2*n_sample/3)]) # testing set contains 1/3 samples
y_fresh = np.asarray(y_new[int(2*n_sample/3):int(n_sample)]) # fresh test set contains 1/3 samples

# y_train
# X_fresh

In [ ]:
# define panel indices
ind_select = [] # index of randomly chosen features for each run
ind_select.append([*range(10)]) # starting with the first 10 features
for i in range(1,n_query+1): #gice Thonak
    inds = [*range(i*n_substitute+9, (i+1)*n_substitute+9)] # try switch 5 each time
    ind_select.append(inds)

ind_select

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [11, 12],
 [13, 14],
 [15, 16],
 [17, 18],
 [19, 20],
 [21, 22],
 [23, 24],
 [25, 26],
 [27, 28],
 [29, 30],
 [31, 32],
 [33, 34],
 [35, 36],
 [37, 38],
 [39, 40],
 [41, 42],
 [43, 44],
 [45, 46],
 [47, 48],
 [49, 50]]

In [ ]:
# for each query, substitute the 5 features with the lowest cost function (delta Acc) with the next 5 unselected featurea
inds = copy.deepcopy(ind_select[0]); # starting with the first 10 features
acc_trains = [] # list of test set accuracy for all subsets, standard resuse
acc_tests = [] # list of test set accuracy for all subsets, standard resuse
acc_freshs = [] # list of test set accuracy for all subsets, Thresholdout
for i in range(n_query):
    part_train = X_train.iloc[:, inds]
    part_test = X_test.iloc[:, inds]
    part_fresh = X_fresh.iloc[:, inds]
    sum_acc_train = 0
    sum_acc_test1 = 0
    sum_acc_test = 0
    sum_acc_fresh = 0
    
    # 3-fold prediction on the panel
    for folds in range(fold): 
        cls1 = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task = t_query*60)
#         cls1 = LinearSVC()
        cls1.fit(part_train, y_train)
        y_test_predict = cls1.predict(part_test); # predicted label of test set
        sum_acc_test1 += sklearn.metrics.accuracy_score(y_test_predict, y_test)
    acc_test1 = sum_acc_test1/fold
    
    # leave One Out calculation of loss function
    acc_test_loss = [] # accuracy of 9 feature subpanels to calculate
    for j in range(10): # each time take out one feature and calculate the new accracies and corresponding loss function
        inds_loss = copy.deepcopy(inds)
        inds_loss.remove(inds[j])
        part_train_loss = X_train.iloc[:, inds_loss]
        part_test_loss = X_test.iloc[:, inds_loss]
        cls2 = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task = t_query*60)
#         cls2 = LinearSVC()
        cls2.fit(part_train_loss, y_train)
        y_test_predict = cls2.predict(part_test_loss); # predicted label of test set
        acc_test2 = sklearn.metrics.accuracy_score(y_test_predict, y_test)
        acc_test_loss.append(acc_test1 - acc_test2)
        
    # find the lowest n loss functions and substitute with new features
    ranking =  sorted(range(len(acc_test_loss)), key=lambda k: acc_test_loss[k]) # index rankning (low to high of the loss functions)
    toRemove = [] # n indices to remove from the current panel
    for k in range(n_substitute):
        toRemove.append(inds[ranking[k]])
    inds = list(set(inds).difference(set(toRemove)))
    for folds in range(fold): 
        cls3 = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task = t_query*60)
#         cls3 = LinearSVC()
        cls3.fit(part_train, y_train)
        y_train_predict = cls3.predict(part_train); # predicted label of training set
        sum_acc_train += sklearn.metrics.accuracy_score(y_train_predict, y_train)
        y_test_predict = cls3.predict(part_test); # predicted label of test set
        sum_acc_test += sklearn.metrics.accuracy_score(y_test_predict, y_test)
        y_fresh_predict = cls3.predict(part_fresh); # predicted label of fresh set
        sum_acc_fresh += sklearn.metrics.accuracy_score(y_fresh_predict, y_fresh)
    acc_train = sum_acc_train/fold
    acc_test = sum_acc_test/fold
    acc_trains.append(acc_train)
    acc_tests.append(acc_test)
    acc_freshs.append(sum_acc_fresh/fold)    
    
    with open('ThresholdOutReport_substitution_2', "a+") as myfile:
        myfile.write("\n query number: "+str(i))
        myfile.write("\n acc_trains = : "+str(acc_trains))
        myfile.write("\n acc_tests = : "+str(acc_tests))
        myfile.write("\n acc_freshs = : "+str(acc_freshs))
        myfile.write("\n acc_test_loss = : "+str(acc_test_loss))
        myfile.write("\n indices of features in panel = : "+str(inds))
        

    #Now insert the next three features
    inds.extend(ind_select[i+1])
        

/home/tliu/.conda/envs/shap/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)


[WARNING] [2019-10-13 20:28:45,130:EnsembleBuilder(1):7fdbf792db4e3d81927ea15ad697c0ec] No models better than random - using Dummy Score!
[WARNING] [2019-10-13 20:28:45,146:EnsembleBuilder(1):7fdbf792db4e3d81927ea15ad697c0ec] No models better than random - using Dummy Score!
Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (119.518821)


/home/tliu/.conda/envs/shap/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/tliu/.conda/envs/shap/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
Process pynisher function call:
Process pynisher function call:
Traceback (most recent call last):
  File "/home/tliu/.conda/envs/shap/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/tliu/.conda/envs/shap/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/tliu/.conda/envs/shap/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/tliu/.conda/envs/shap/lib/python3.7/site-packages/pynisher/limit_function_call.py", lin

[WARNING] [2019-10-13 20:29:02,463:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2019-10-13 20:29:02,463:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[20:29:02] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[20:29:02] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[20:29:02] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[20:29:02] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[20:29:02] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[20:29:02] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[20:29:02] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[20:29:02] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[20:29:02] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[20:29:02] /workspace/src/gbm/gbtree.cc:492: drop 0 trees, w

/home/tliu/.conda/envs/shap/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/tliu/.conda/envs/shap/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/tliu/.conda/envs/shap/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/tliu/.conda/envs/shap/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/tliu/.conda/envs/shap/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/tliu/.conda/envs/shap/li

In [ ]:
# With ThresholdOut
# for each query, substitute the 3 features with the lowest cost function (delta Acc) with the next 5 unselected featurea
inds = copy.deepcopy(ind_select[0]); # starting with the first 10 features
acc_trains = [] # list of test set accuracy for all subsets, standard resuse
acc_tests = [] # list of test set accuracy for all subsets, standard resuse
acc_freshs = [] # list of test set accuracy for all subsets, Thresholdout
for i in range(n_query):
    part_train = X_train.iloc[:, inds]
    part_test = X_test.iloc[:, inds]
    part_fresh = X_fresh.iloc[:, inds]
    sum_acc_train = 0
    sum_acc_test1 = 0
    sum_acc_test = 0
    sum_acc_fresh = 0
    
    # 3-fold prediction on the panel
    for folds in range(fold): 
        cls1 = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task = t_query*60)
#         cls1 = LinearSVC()
        cls1.fit(part_train, y_train)
        y_test_predict = cls1.predict(part_test); # predicted label of test set
        sum_acc_test1 += sklearn.metrics.accuracy_score(y_test_predict, y_test)
    acc_test1 = sum_acc_test1/fold
    
    # leave One Out calculation of loss function
    acc_test_loss = [] # accuracy of 9 feature subpanels to calculate
    for j in range(10): # each time take out one feature and calculate the new accracies and corresponding loss function
        inds_loss = copy.deepcopy(inds)
        inds_loss.remove(inds[j])
        part_train_loss = X_train.iloc[:, inds_loss]
        part_test_loss = X_test.iloc[:, inds_loss]
        cls2 = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task = t_query*60)
#         cls2 = LinearSVC()
        cls2.fit(part_train_loss, y_train)
        y_train_predict = cls2.predict(part_train_loss); # predicted label of test set
        acc_train2 = sklearn.metrics.accuracy_score(y_train_predict, y_train)
        y_test_predict = cls2.predict(part_test_loss); # predicted label of test set
        acc_test2 = sklearn.metrics.accuracy_score(y_test_predict, y_test)
        if acc_train2 - acc_test2 < threshold1 + np.random.laplace(0, noise1): #apply thresholdout to query output
            acc_test2 = acc_train2
        else:
            acc_test2 = acc_test2 + np.random.laplace(0, noise1)
        acc_test_loss.append(acc_test1 - acc_test2)
        
    # find the lowest n loss functions and substitute with new features
    ranking =  sorted(range(len(acc_test_loss)), key=lambda k: acc_test_loss[k]) # index rankning (low to high of the loss functions)
    toRemove = [] # n indices to remove from the current panel
    for k in range(n_substitute):
        toRemove.append(inds[ranking[k]])
    inds = list(set(inds).difference(set(toRemove)))
    
    for folds in range(fold): 
        cls3 = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task = t_query*60)
#         cls3 = LinearSVC()
        cls3.fit(part_train, y_train)
        y_train_predict = cls3.predict(part_train); # predicted label of training set
        sum_acc_train += sklearn.metrics.accuracy_score(y_train_predict, y_train)
        y_test_predict = cls3.predict(part_test); # predicted label of test set
        sum_acc_test += sklearn.metrics.accuracy_score(y_test_predict, y_test)
        y_fresh_predict = cls3.predict(part_fresh); # predicted label of fresh set
        sum_acc_fresh += sklearn.metrics.accuracy_score(y_fresh_predict, y_fresh)
    acc_train = sum_acc_train/fold
    acc_test = sum_acc_test/fold
    acc_trains.append(acc_train)
    acc_tests.append(acc_test)
    acc_freshs.append(sum_acc_fresh/fold)    
    
    with open('ThresholdOutReport_substitution_2_TO', "a+") as myfile:
        myfile.write("\n query number: "+str(i))
        myfile.write("\n acc_trains = : "+str(acc_trains))
        myfile.write("\n acc_tests = : "+str(acc_tests))
        myfile.write("\n acc_freshs = : "+str(acc_freshs))
        myfile.write("\n acc_test_loss = : "+str(acc_test_loss))
        myfile.write("\n indices of features in panel = : "+str(inds))
        

    #Now insert the next three features
    inds.extend(ind_select[i+1])
    
    


In [ ]:
# import pickle
# # save the synthetic data 
# dump_file = 'ThresholdoutData_X.dump.pkl'
# with open(dump_file, 'wb') as f:
#     pickle.dump(X, f)
#     f.close()
# dump_file = 'ThresholdoutData_y.dump.pkl'
# with open(dump_file, 'wb') as f:
#     pickle.dump(y, f)
#     f.close()


In [ ]:
# load file
import pickle
# f = open('ThresholdoutData_1.dump.pkl', "rb")
# a = pickle.load(f)
# f.close()
# X = a[0] # data part of the dataset

f = open('ThresholdoutData_X.dump.pkl', "rb")
X = pickle.load(f)
f.close()
f = open('ThresholdoutData_y.dump.pkl', "rb")
y = pickle.load(f)
f.close()
X_train =X[0:100] # training set contains 100 samples
X_test =X[100:200] # testing set contains 100 samples
X_fresh =X[200:300] # fresh test set contains 100 samples
y_train = np.asarray(y[0:100]) # training set contains 100 samples
y_test = np.asarray(y[100:200]) # testing set contains 100 samples
y_fresh = np.asarray(y[200:300]) # fresh test set contains 100 samples
X_train = pd.DataFrame(X_train) # transfer to pandas dataframe
X_test = pd.DataFrame(X_test)
X_fresh = pd.DataFrame(X_fresh)
